# Gramáticas basadas en rasgos

[Documentación](https://www.nltk.org/howto/featstruct.html)

In [1]:
import nltk

Las gramáticas se pueden enriquecer con el uso de rasgos. En una gramática con rasgos, los rasgos se heredan de las entradas léxicas a los nodos superiores. Las reglas especifican los rasgos que sus nodos hijos deben compartir.

Temas de Python que me parece que vamos a tocar en esta clase:

- Diccionarios
- Conjuntos
- Funciones
- Herencia
- `*args`
- Métodos de NLTK
- Función `lambda`

## Estructuras de rasgos

### Representación con diccionarios

In [ ]:
# representar rasgos como diccionarios
# ampliar estructura con nuesvos atributos

### Representación como conjuntos

### Representación con NLTK

In [6]:
fs1 = nltk.FeatStruct(TENSE='past', CHICHO=True)

In [7]:
fs2 = nltk.FeatStruct(CAT='V')

In [29]:
fs3 = nltk.FeatStruct(CAT='V', TENSE='present')

In [36]:
fs3 = nltk.FeatStruct("""[CAT=V, TENSE=present]""")

In [40]:
# explicar representación de inespecificidad

### Representación con un grafo

In [39]:
# armar

## Estructuras de rasgos como funciones

Acá podemos mostrar:
- función que toma un atributo y devuelve un valor
- función que toma un path de atributos y devuelve el valor siguiendo ese path

Para esto -> armar una clase que herede de FeatStruct e implementar un método que recorra la estructura. Como argumento de la función usar `*args`.

Podemos aprovechar y ver el paralelismo con la noción de herencia, que tiene una procedencia computacional.

## Unificación

In [28]:
print(fs1.unify(fs3, trace=1))


Unification trace:
   / [+CHICHO, TENSE='past']
  |\ [CAT='V', TENSE='present']
  |
  | Unify feature: TENSE
  |    / 'past'
  |   |\ 'present'
  |   |
  X   X <-- FAIL
None


Acá mostrar:

- unificación exitosa
- unificación fallida
- unificación vs. unión: antes vimos que las estructuras se pueden representar como conjuntos. Este es un buen momento para mostrar que la unificación y la unión funcionan distinto y que la única situación en la que funcionan igual es con rasgos atómicos.

## Subsunción

In [16]:
fs2.subsumes(fs3)

True

## Rasgos atómicos y complejos

In [18]:
fs = nltk.FeatStruct(CAT='V', NUM='plu')

In [19]:
fs

[CAT='V', NUM='plu']

In [20]:
fs['CONCORD'] = fs

In [21]:
fs

(1)[CAT='V', CONCORD->(1), NUM='plu']

In [22]:
fs.cyclic()

True

In [27]:
# retomar feature sharing
# recordar que compartir rasgos no es lo mismo que tener duplicada la misma información
# mostrar con equal_values y hablar sobre reentrancia

In [26]:
fs.equal_values(fs, check_reentrance=False)

True

Acá también podríamos mostrar que una estructura que comparte rasgos es más específica que una que no los comparte aunque tiene esepecificados los mismos valores para los mismos atributos.

## Construcción de una gramática

In [ ]:
nltk.data.show_cfg('gramaticas/GramaticaDeRasgos.fcfg')

In [ ]:
sentence = 'los chicas caminan'
tokens = sentence.split()
print(sentence)
print(type(sentence))
print(tokens)
print(type(tokens))

In [ ]:
from nltk import load_parser
cp = load_parser('gramaticas/GramaticaDeRasgos.fcfg', trace=2)
for tree in cp.parse(tokens):
     print(tree)

## Uso de rasgos para significado

Los rasgos pueden utilizarse a su vez para construir una representación semántica de las oraciones.

En semántica formal existe una función particular, que se conoce con el nombre de función interpretación, y que se anota con corchetes dobles. La función interpretación devuelve por cada expresión lingüística su denotación. Las denotaciones pueden ser de dos tipos: 

- elementos atómicos (típicamente objetos o proposiciones, pero también hay otras ontologías que incluyen mundos posibles, eventos y tiempos, entre otras cosas)
- funciones. 

El uso de rasgos para dar cuenta del significado consiste en que la función denotación sea el valor de un rasgo semántico.

En el [libro de NLTK](https://www.nltk.org/book/ch10.html) y en la [documentación de NLTK](http://nltk.sourceforge.net/doc/en/ch11.html) hay información sobre cómo implementar esto mediante una gramática de rasgos en NLTK.


Las funciones equivalen a conjuntos y se expresan en el llamado cálculo lambda. 

\x. x fuma

Esta es una función que toma un x y devuelve verdadero si x fuma y falso si x no fuma. En términos de conjuntos equivale al conjunto de todos los fumadores (ténicamente equivale al conjunto característico de todos los fumadores, que es el que devuelve verdadero si x fuma y falso si x no fuma).

Hay dos operaciones básicas de cálculo lambda que son particularmente relevantes (una tercera no tuvo tanta repercusión en la semántica formal):

- Conversión alpha (o reducción alpha): cambiar el nombre de una variable y, conjuntamente, el de todas las variables ligadas con ella.
`[\x. x fuma] = [\y. y fuma] = [\z. z fuma] = ...`
- Conversión lambda (o reducción beta): cuando combinamos una función con un argumento, eliminar el prefijo lambda y reemplazar todas las ocurrencias de la variable que introduce ese prefijo por el argumento.

- `[\x. x fuma](cata) = cata fuma`
- `[\f. f](\x. x fuma) = \x. x fuma`
- `[\f. [\g. [\x. g(x)=f(x)=1]]](\x. x fuma)(\x. x baila) = [\x. [\x. x fuma](x)=[\x. x baila](x)=1] = [\x x fuma y x baila]` 

Las interpretación semántica de las expresiones lingüísticas se da a partir del significado de sus partes y su combinación mediante reglas que dependen de la forma del árbol y de los tipos de las funciones. Las reglas más frecuentes son: 

- Aplicación funcional: Si un nodo A domina a dos nodos B y C tales que B es una función cuyo dominio incluye a C, entonces [[A]]=[[B]]([[C]])
- Modificación de predicados: Si un nodo A domina a dos nodos B y C tales que los dos nodos son funciones que van del dominio de los individuos al dominio de los valores de verdad, entonces [[A]] = \x. [[B]]=[[C]]=1

Las fcfg implementan la función intepretación como valor de un rasgo semántico y reemplazando las reglas que dependen de la forma del árbol directamente por restricciones en las reglas de reescritura. Estas restricciones consisten básicamente en la unificación mediante variables idénticas.

In [ ]:
nltk.data.show_cfg('gramaticas/pruebasemantica.fcfg')

In [ ]:
sents = ['Cata fuma']
grammar = 'gramaticas/pruebasemantica.fcfg'
for results in nltk.interpret_sents(sents, grammar):
    for (synrep, semrep) in results:
             print(synrep)

## Semántica eventiva

Un tipo de semántica formal que hoy ha ganado mucha popularidad es la semántica eventiva. La semántica eventiva concibe el significado de las oraciones como cuantificación sobre eventos: 

- Juan le dio ayer el libro a Pedro en la casa.
- Existe un e tal que Agente(e, Juan) & Tema(e, el libro) & Meta(e, Pedro) & en(e, la casa) & Pasado(e)

La semántica neodavidsoniana permite dispensar de grandes listas de subclases (verbos transitivos, verbos intransitivos, verbos ditransitivos, verbos bivalentes, verbos impersonales, verbos que toman distintos tipos de sintagmas preposicionales) y simplificar, en consecuencia, el léxico. Esto se hace al costo de dar una denotación más compleja a las entradas léxicas (que sin embargo, se puede automatizar si se tienen listas léxicas como las de freeling) y a las reglas de reescritura. Este tipo de enfoques propenden a sobregenerar, pero, naturalmente, este no es un problema si lo que nos interesa es parsear oraciones.

In [ ]:
sents = ['Cata dio Ficciones a Chafa']
grammar = 'gramaticas/semantica_eventiva_base.fcfg'
for results in nltk.interpret_sents(sents, grammar):
    for (synrep, semrep) in results:
             print(synrep)